In [1]:
import os
import copy
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import itertools

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential

import hls4ml
from hls4ml.converters.keras_to_hls import parse_default_keras_layer
from hls4ml.model.attributes import ConfigurableAttribute, TypeAttribute
from hls4ml.model.types import FixedPrecisionType, RoundingMode, SaturationMode
from hls4ml.model.attributes import Attribute

import h5py

import qkeras as qk
from qkeras.estimate import print_qstats
from qkeras.utils import model_quantize
from qkeras.utils import quantized_model_dump
from qkeras import QActivation, QDense, QConv2DBatchnorm

# Source the Vivado path
os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']

np.random.seed(0)
tf.random.set_seed(0)


2025-06-30 17:34:57.016789: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-30 17:34:57.297278: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-30 17:34:57.313248: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-06-30 17:34:57.313328: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

/home/aelabd/RHEED/hls4ml/hls4ml/converters/__init__.py:29: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


Helper functions

In [2]:
def print_quantization_info(model):
    for layer in model.layers:
        print(f"Layer Name: {layer.name}")
        print(f"Type: {layer.__class__.__name__}")
        
        # Helper function to handle both quantizer objects and config dicts
        def process_quantizer(quantizer, prefix=""):
            if quantizer:
                if isinstance(quantizer, dict):
                    # Handle dictionary config
                    class_name = quantizer.get("class_name", "UnknownQuantizer")
                    config = quantizer.get("config", {})
                else:
                    # Handle object with potential get_config()
                    class_name = quantizer.__class__.__name__
                    config = quantizer.get_config() if hasattr(quantizer, "get_config") else {}
                
                print(f"  {prefix}Quantizer: {class_name}")
                print(f"  {prefix}Config: {config}")
            else:
                print(f"  No {prefix}Quantizer")

        # Check for QKeras layers with kernel/bias quantizers
        if isinstance(layer, (qk.QDense, qk.QConv2D, qk.QConv1D, 
                            qk.QConv2DTranspose, qk.QDepthwiseConv2D)):
            # Kernel quantizer
            process_quantizer(layer.kernel_quantizer, "Kernel ")
            
            # Bias quantizer
            process_quantizer(layer.bias_quantizer, "Bias ")
            
            # Activation quantizer
            activation = layer.activation
            if activation:
                if isinstance(activation, dict) or hasattr(activation, "get_config"):
                    process_quantizer(activation, "Activation ")
                else:
                    print(f"  Activation: {activation} (Not Quantized)")
            else:
                print("  No Activation")

        # Check for QActivation layers
        elif isinstance(layer, qk.QActivation):
            process_quantizer(layer.quantizer, "Activation ")
        
        print("-" * 50)

def generate_binary_numbers(n_bits):
    return np.array([''.join(bits) for bits in itertools.product('01', repeat=n_bits)])

def bin_to_int(bin_str):
    int_val = 0
    for i, bit in enumerate(bin_str[::-1]):
        int_val += int(bit)*(2**i)
    return int_val

def bin_to_frac(bin_str):
    frac_val = 0
    for i, bit in enumerate(bin_str):
        frac_val += int(bit)*(2**(-i-1))
    return frac_val

def frac_to_bin(frac, n_bits=8):
    bin_str = ""
    for i in range(n_bits):
        if frac >= 2**(-i-1):
            bin_str += "1"
            frac -= 2**(-i-1)
        else:
            bin_str += "0"
    return bin_str

def hex_str_to_float_frac(hex_str):
    int_num = int(hex_str.replace("\n", "").strip(), 16)
    int_bin_str = np.binary_repr(int_num)
    float_frac = bin_to_frac(int_bin_str)
    return float_frac
    

# 1. Load Keras model

In [3]:
KERAS_DIR = "/home/aelabd/RHEED/keras_models"
RTL_DIR = "/home/aelabd/RHEED/rtl_models"

def dice_loss(y_true, y_pred, delta=0.6):
    error = y_true - y_pred
    is_small = tf.abs(error) <= delta
    squared_loss = 0.5 * tf.square(error)
    linear_loss = delta * (tf.abs(error) - 0.5 * delta)
    return tf.reduce_mean(tf.where(is_small, squared_loss, linear_loss))

fpath_model_keras = os.path.join(KERAS_DIR, "model.keras")
with tf.keras.utils.custom_object_scope({'dice_loss': dice_loss,
                                         'QConv2DBatchnorm': QConv2DBatchnorm,
                                         'QActivation': QActivation,
                                         'QDense': QDense
                                         }):
        model = tf.keras.models.load_model(fpath_model_keras)

print_quantization_info(model)

2025-06-30 17:35:06.235764: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-06-30 17:35:06.235978: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-06-30 17:35:06.236038: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2025-06-30 17:35:06.236090: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2025-06-30 17:35:06.236127: W tensorf

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Layer Name: q_conv2d_batchnorm_5
Type: QConv2DBatchnorm
  Kernel Quantizer: quantized_bits
  Kernel Config: {'bits': 8, 'integer': 0, 'symmetric': 0, 'alpha': 1, 'keep_negative': True, 'use_stochastic_rounding': False, 'qnoise_factor': 1.0}
  Bias Quantizer: quantized_bits
  Bias Config: {'bits': 8, 'integer': 0, 'symmetric': 0, 'alpha': 1, 'keep_negative': True, 'use_stochastic_rounding': False, 'qnoise_factor': 1.0}
  Activation: <function linear at 0x7feef6eb4430> (Not Quantized)
--------------------------------------------------
Layer Name: q_activation_8
Type: QActivation
  Activation Quantizer: quantized_relu
  Activation Config: {'bits': 8, 'integer': 2, 'use_sigmoid': 0, 'negative_slope': 0.0, 'use_stochastic_rounding': False, 'relu_upper_bound': None, 'qnoise_factor': 1.0}
--------------------------------------------------
Layer Name: max_pooling2d_5
Type: MaxPooling2D
--------------------------------------------------
Layer Name: q_conv2d_batchnorm_6
Type: QConv2DBatchnorm
  

# 2. Load testbench input data

In [4]:
IN_ROWS = 100
IN_COLS = 160
OUT_ROWS = 48
OUT_COLS = 48
NUM_CROPS = 1
data_dir = f"tb_data_Mono8/{IN_ROWS}x{IN_COLS}_to_{OUT_ROWS}x{OUT_COLS}x{NUM_CROPS}"

CROP_X0 = [0, 13, 112]
CROP_Y0 = [0, 1, 52]
input_data = {}
for y0 in CROP_Y0:
    input_data[f"y1_{y0}"] = {}
    for x0 in CROP_X0:
        crop_data = []
        fpath = os.path.join(data_dir, f"Y1_{y0}/X1_{x0}/HDL_cropnorm_out.txt")
        with open(fpath, "r") as f:
            for line in f.readlines():

                line_float = np.zeros((OUT_COLS,))
                for i, hex_str in enumerate(line.split(" ")):
                    hex_str_clean = hex_str.replace("\n", "").replace(" ", "")
                    if len(hex_str_clean) < 1: continue
                    line_float[i] = hex_str_to_float_frac(hex_str_clean)

                crop_data.append(line_float)
        input_data[f"y1_{y0}"][f"x1_{x0}"] = np.expand_dims(np.expand_dims(np.array(crop_data), 0), 3)

# 3. Generate predictions

# NOTE: Output quantization is <8,2> not <8,0>

In [ ]:
def float_to_ap_fixed(value, X, Y):
    """
    Converts a float to a binary string representing a signed ap_fixed<X,Y> number.
    
    Parameters:
        value (float): The float value to convert.
        X (int): Total number of bits.
        Y (int): Number of integer bits (including sign bit).
    
    Returns:
        str: Binary string of length X.
    """
    # Number of fractional bits
    F = X - Y

    # Compute scaled integer
    scaled = int(round(value * (2 ** F)))

    # Calculate min and max representable values
    min_val = -(2 ** (X - 1))
    max_val = (2 ** (X - 1)) - 1

    # Clip to representable range
    if scaled < min_val:
        scaled = min_val
    elif scaled > max_val:
        scaled = max_val

    # Convert to two's complement binary
    if scaled < 0:
        scaled = (1 << X) + scaled  # two's complement

    bin_str = format(scaled, f'0{X}b')  # zero-padded binary string of width X
    return bin_str

print(float_to_ap_fixed(3.25, 22, 11))   # ap_fixed<8,4> => 4 int bits, 4 fractional bits
# Output: '00110100'  (3.25 * 16 = 52 => 0b00110100)

print(float_to_ap_fixed(-2.5, 22, 11))
# Output: '11000000'  (-2.5 * 16 = -40 => two's complement => 0b11000000)

0000000001101000000000
1111111110110000000000


In [18]:
pred_data = {}
output_data = {}
for y0 in CROP_Y0:
    pred_data[f"y1_{y0}"] = {}
    for x0 in CROP_X0:
        print(f"y0={y0}, x0={x0}")
        pred = model.predict(input_data[f"y1_{y0}"][f"x1_{x0}"])[0]
        pred_data[f"y1_{y0}"][f"x1_{x0}"] = pred
        fpath_txt = os.path.join(data_dir, f"Y1_{y0}/X1_{x0}/QKeras_pred_ap_fixed_22_11.txt")
        with open(fpath_txt, "w") as f:
            for val in pred:
                bin_str = float_to_ap_fixed(val, 22, 11)
                int_num = bin_to_int(bin_str)
                hex_str = hex(int_num).replace("0x", "").upper()
                if len(hex_str) < 2: hex_str = "0" + hex_str
                f.write(f"{hex_str} ")
                print(f"val: {val}, bin_str: {bin_str}")

y0=0, x0=0
1/1 [==============================] - 0s 411ms/step
val: 0.451171875, bin_str: 0000000000001110011100
val: 0.5185546875, bin_str: 0000000000010000100110
val: -0.63134765625, bin_str: 1111111111101011110011
val: -0.43115234375, bin_str: 1111111111110010001101
val: -0.12060546875, bin_str: 1111111111111100001001
y0=0, x0=13
1/1 [==============================] - 0s 18ms/step
val: 0.45068359375, bin_str: 0000000000001110011011
val: 0.51904296875, bin_str: 0000000000010000100111
val: -0.63720703125, bin_str: 1111111111101011100111
val: -0.43017578125, bin_str: 1111111111110010001111
val: -0.11669921875, bin_str: 1111111111111100010001
y0=0, x0=112
1/1 [==============================] - 0s 18ms/step
val: 0.451171875, bin_str: 0000000000001110011100
val: 0.5185546875, bin_str: 0000000000010000100110
val: -0.63134765625, bin_str: 1111111111101011110011
val: -0.43115234375, bin_str: 1111111111110010001101
val: -0.12060546875, bin_str: 1111111111111100001001
y0=1, x0=0
1/1 [========